# An aspherical $5$-manifold: $S_2 \times \text{m160(3,1)}$

In [1]:
import twisted_l2
import regina
import snappy

In [2]:
mfd = snappy.Manifold("m160(3,1)")
m160_tri = regina.Triangulation3(mfd.filled_triangulation())

In [3]:
s2_fl = twisted_l2.surface_face_lattice(2)

In [4]:
m160_fl = twisted_l2.regina_tri_to_face_lattice(m160_tri, ideal=False)

In [5]:
twisted_l2.load_hap()

true

In [6]:
s2 = twisted_l2.cw_complex(s2_fl)
m160 = twisted_l2.cw_complex(m160_fl)

In [7]:
m5 = libgap.DirectProduct(s2, m160)

In [8]:
[twisted_l2.gap_member(m5, "nrCells")(j) for j in range(6)]

[324, 3060, 10032, 14784, 9984, 2496]

In [9]:
[twisted_l2.gap_member(s2, "nrCells")(j) for j in range(6)]

[6, 16, 8, 0, 0, 0]

In [10]:
[twisted_l2.gap_member(m160, "nrCells")(j) for j in range(6)]

[54, 366, 624, 312, 0, 0]

In [11]:
[twisted_l2.gap_member(m5, "nrCells")(j) for j in range(6)]

[324, 3060, 10032, 14784, 9984, 2496]

## The fiber of m160(3,1)

From "_Fibred and Virtually Fibred Hyperbolic 3-Manifolds in the Censuses_" by J. O. Button,\
we know that m160(3,1) is a fibered closed hyperbolic 3-manifold with first Betti number $1$ (see also `betti-numbers-F.ipynb`).\
If $F$ is the fiber, then $S_2 \times F$ is the fiber of m5.

We will show:
- $F$ has Alexander polynomial of degree $4$
- hence, $F$ has genus $2$
- hence, $\chi(F) = -2$
- hence, $\chi(S_2 \times F) = 4$

In [53]:
from sage.groups.finitely_presented import wrap_FpGroup

In [54]:
G_m160 = wrap_FpGroup(libgap.FundamentalGroup(m160))

In [64]:
ab_m160 = G_m160.abelianization_map(); ab_m160

Group morphism:
  From: Finitely presented group < f1, f3 | f3*f1*(f3^-1*(f1^-1*f3)^3*f1^-1*f3^-1)^2*f1, (f3^-1*f1^-1)^2*f3*((f1^-1*f3)^2*f1^-1)^3*f3*f1^-1 >
  To:   Finitely presented group < f1, f2 | f1^3, f2^-1*f1^-1*f2*f1 >

In [66]:
[ab_m160(g) for g in G_m160.gens()]

[f2, f1^2*f2^2]

In [71]:
R.<t> = PolynomialRing(QQ)
am = G_m160.alexander_matrix(im_gens=[t, t^2]) 
gcd(am.minors(1))

(t^4 - t^3 + 3*t^2 - t + 1)/t^6

As we wanted, the degree is $4$.

## Identifying the class of the fiber in $\mathrm{Hom}(G; \mathbb Z)$

In [13]:
%time chain_complex = twisted_l2.equivariant_cc(m5)

CPU times: user 6.28 s, sys: 65.8 ms, total: 6.34 s
Wall time: 6.27 s


In [14]:
G = chain_complex.group

In [15]:
G.AbelianInvariants()

[ 0, 0, 0, 0, 0, 3 ]

In [16]:
ab = G.MaximalAbelianQuotient()

In [17]:
ab.Image()

<fp group of size infinity on the generators [ f1, f2, f3, f4, f5, f6 ]>

In [18]:
ab.Image().RelatorsOfFpGroup()

[ f1^3, f2^-1*f1^-1*f2*f1, f3^-1*f1^-1*f3*f1, f3^-1*f2^-1*f3*f2, f4^-1*f1^-1*f4*f1, f4^-1*f2^-1*f4*f2, f4^-1*f3^-1*f4*f3, f5^-1*f1^-1*f5*f1, f5^-1*f2^-1*f5*f2, f5^-1*f3^-1*f5*f3, f5^-1*f4^-1*f5*f4, f6^-1*f1^-1*f6*f1, f6^-1*f2^-1*f6*f2, f6^-1*f3^-1*f6*f3, f6^-1*f4^-1*f6*f4, f6^-1*f5^-1*f6*f5 ]

This result is **non-deterministic**. In general, the last $\mathrm{rk}(G)$ generators (\*) will generate a free abelian subgroup\
and the others will generate the torsion subgroup. Here is how to extract the free abelian generators without assuming (\*):

In [31]:
tors_ab_gen = {abs(j) for rel in ab.Image().RelatorsOfFpGroup()
                        for word in (rel.LetterRepAssocWord().sage(),) # cache the Tietze word 
                        for j in word                                  # to avoid calling GAP every time
                        if len(set(word)) != 4} # discard the commutators
free_ab_gen = {j for j in range(1,len(ab.Image().GeneratorsOfGroup())+1) if j not in tors_ab_gen}
G_to_Z5 = {g: j for j, g in enumerate(sorted(free_ab_gen))}
G_to_Z5

{2: 0, 3: 1, 4: 2, 5: 3, 6: 4}

In [24]:
num_edges = twisted_l2.gap_member(m5, "nrCells")(1); num_edges

3060

In [25]:
edge_to_word = twisted_l2.gap_member(G, "edgeToWord")

In [34]:
edge_words = [ab.Image(edge_to_word(x)).UnderlyingElement().LetterRepAssocWord().sage() for x in range(1, num_edges+1)]

In [37]:
def word_to_tuple(w):
    t = [0,0,0,0,0]
    for j in w:
        if abs(j) in G_to_Z5:
            t[G_to_Z5[abs(j)]] += sign(j)
    return tuple(t)

In [38]:
edge_coords = [word_to_tuple(w) for w in edge_words]

The following set appears to be the same across different runs.\
We assume it is so, because GAP internals are hard to save to a file in a robust way (no `pickle`).

In [44]:
sorted(set(edge_coords))

[(-3, 0, 0, 0, 0),
 (-2, 0, 0, 0, 0),
 (-1, 0, 0, 0, 0),
 (0, -1, 1, 0, 0),
 (0, 0, 0, -1, 1),
 (0, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 0, 1, 0),
 (0, 0, 1, 0, 0),
 (0, 1, 0, 0, 0),
 (1, 0, 0, 0, 0),
 (2, 0, 0, 0, 0),
 (3, 0, 0, 0, 0)]

In [47]:
def e5_to_e3(e5):
    """Maps an edge in the direct product to the corresponding edge in the m160(3,1)"""
    q = twisted_l2.gap_member(m5, "pair2quad")[1][e5].sage()
    i,j,x,y = q[0]-1, q[1]-1, q[2]-1, q[3]-1
    # These four numbers are:
    #     - i: dimension of the projection onto s2   (0 or 1)
    #     - j: dimension of the projection onto m160 (0 or 1)
    #     - x: edge of s2   (if i == 1)
    #     - y: edge of m160 (if j == 1)
    if i == 0:
        return y
    return None

Here we find all words associated to edges of the form $e\times \{p\}$ (where $p$ is a point of m160(3,1) and $e$ is an edge of $S_2$).

In [48]:
s2_edges = set()
for j, tup in enumerate(edge_coords):
    e3 = e5_to_e3(j)
    if e3 is None:
        s2_edges.add(tup)

In [49]:
len(s2_edges)

7

In [50]:
s2_edges

{(0, -1, 1, 0, 0),
 (0, 0, 0, -1, 1),
 (0, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 0, 1, 0),
 (0, 0, 1, 0, 0),
 (0, 1, 0, 0, 0)}

Fix a point $p$ in m160(3,1). Since both the class of the fiber in `m5` and the cohomology class `(1,0,0,0,0)` kill every _loop_ in $S_2\times\{p\}$, they are equal.\
Therefore, we predict that our program will give $\chi(S_2\times F) = -4$ as the twisted $L^2$-Euler characteristic of `(1,0,0,0,0)`.

## Computation for v = (1,0,0,0,0)

In [72]:
%time cc = twisted_l2.get_differentials(chain_complex)

CPU times: user 1.8 s, sys: 6.73 ms, total: 1.81 s
Wall time: 1.82 s


Here we can load a precomputed chain complex:

In [89]:
# twisted_l2.save_to_file("data/m5-opti-2.json", group=G, cc=cc)
#
# a more complicated chain complex:
# G, cc = twisted_l2.load_from_file("data/m5.json")
G, cc = twisted_l2.load_from_file("data/m5-opti.json")

In [90]:
[cc[j].dimensions() for j in range(5)]

[(6, 1), (11, 6), (11, 11), (6, 11), (1, 6)]

In [91]:
G.abelian_invariants()

(0, 0, 0, 0, 0, 3)

In [92]:
twisted_l2.configs.LogOptions.LEVEL = twisted_l2.INFO

We use `exps = (1)` because $G$ has large rank, so increasing the nilpotency class makes $|L|$ grow very quickly.\
It suffices to take `n = 4` for the valuations to stabilize.\
The degrees of the Laplacians for the `m5-opti` chain complex are

    [2, 18, 52, 52, 18, 2]
    
and the result is exactly $4$, as desired.

In [100]:
%time twisted_l2.characteristic(G, [1,0,0,0,0], cc, 4, (1,))

=========== Dimension 0 ===========

Lift: F[1]
Maximum valuation of entries = 2, should expand matrix by 2, expanding by 2 instead
Size of Fin: 32
|L| = 16
Constructing matrix over Q[L]...
Computing rank...
Dimensions of N: (32, 32)
Rank: 2.0 (rounds up to 2)
Valuation: 0.0
Degree: 2.0

=========== Dimension 1 ===========

Lift: F[1]
Maximum valuation of entries = 8, should expand matrix by 48, expanding by 4 instead
Size of Fin: 32
|L| = 16
Constructing matrix over Q[L]...
Computing rank...
Dimensions of N: (384, 384)
Rank: 21.0 (rounds up to 21)
Valuation: 3.0
Degree: 18.0

=========== Dimension 2 ===========

Lift: F[1]
Maximum valuation of entries = 9, should expand matrix by 99, expanding by 4 instead
Size of Fin: 32
|L| = 16
Constructing matrix over Q[L]...
Computing rank...
Dimensions of N: (704, 704)
Rank: 28.0 (rounds up to 28)
Valuation: 16.0
Degree: 52.0

=========== Dimension 3 ===========

Lift: F[1]
Maximum valuation of entries = 9, should expand matrix by 99, expanding 

4

In [99]:
%time twisted_l2.characteristic(G, [1,0,0,0,0], cc, 5, (1,))

=========== Dimension 0 ===========

Lift: F[1]
Maximum valuation of entries = 2, should expand matrix by 2, expanding by 2 instead
Size of Fin: 32
|L| = 16
Constructing matrix over Q[L]...
Computing rank...
Dimensions of N: (32, 32)
Rank: 2.0 (rounds up to 2)
Valuation: 0.0
Degree: 2.0

=========== Dimension 1 ===========

Lift: F[1]
Maximum valuation of entries = 8, should expand matrix by 48, expanding by 5 instead
Size of Fin: 32
|L| = 16
Constructing matrix over Q[L]...
Computing rank...
Dimensions of N: (480, 480)
Rank: 27.0 (rounds up to 27)
Valuation: 3.0
Degree: 18.0

=========== Dimension 2 ===========

Lift: F[1]
Maximum valuation of entries = 9, should expand matrix by 99, expanding by 5 instead
Size of Fin: 32
|L| = 16
Constructing matrix over Q[L]...
Computing rank...
Dimensions of N: (880, 880)
Rank: 39.0 (rounds up to 39)
Valuation: 16.0
Degree: 52.0

=========== Dimension 3 ===========

Lift: F[1]
Maximum valuation of entries = 9, should expand matrix by 99, expanding 

4